## Prudential Data Engineering (Kaggle Dataset)

 
        

There are three types of variables in the dataset, categorical, continuous, and discrete.  The first step will be to ensure that the features are normalized, and correct for truncated data.  The second step will be to create dummy variables for our categorical data.  This will be done such that a dummy is created for any category that has more than 20 observations.

In [2]:
from numpy import loadtxt, zeros, ones, array, linspace, logspace
from pylab import scatter, show, title, xlabel, ylabel, plot, contour
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
traindf = pd.read_csv("train.csv")
descrip = traindf.describe()

Categorical = ['Product_Info_1', 'Product_Info_2', 'Product_Info_3', 'Product_Info_5', 
'Product_Info_6', 'Product_Info_7', 'Employment_Info_2', 'Employment_Info_3', 
'Employment_Info_5', 'InsuredInfo_1', 'InsuredInfo_2', 'InsuredInfo_3', 'InsuredInfo_4', 
'InsuredInfo_5', 'InsuredInfo_6', 'InsuredInfo_7', 'Insurance_History_1', 
'Insurance_History_2', 'Insurance_History_3', 'Insurance_History_4', 
'Insurance_History_7', 'Insurance_History_8', 'Insurance_History_9', 'Family_Hist_1', 
'Medical_History_2', 'Medical_History_3', 'Medical_History_4', 'Medical_History_5', 
'Medical_History_6', 'Medical_History_7', 'Medical_History_8', 'Medical_History_9', 
'Medical_History_11', 'Medical_History_12', 'Medical_History_13', 'Medical_History_14', 
'Medical_History_16', 'Medical_History_17', 'Medical_History_18', 'Medical_History_19', 
'Medical_History_20', 'Medical_History_21', 'Medical_History_22', 'Medical_History_23', 
'Medical_History_25', 'Medical_History_26', 'Medical_History_27', 'Medical_History_28',
'Medical_History_29', 'Medical_History_30', 'Medical_History_31', 'Medical_History_33', 
'Medical_History_34', 'Medical_History_35', 'Medical_History_36', 'Medical_History_37', 
'Medical_History_38', 'Medical_History_39', 'Medical_History_40', 'Medical_History_41']

Continuous = ['Product_Info_4', 'Ins_Age', 'Ht', 'Wt', 'BMI', 'Employment_Info_1', 
'Employment_Info_4', 'Employment_Info_6', 'Insurance_History_5', 'Family_Hist_2', 
'Family_Hist_3', 'Family_Hist_4', 'Family_Hist_5']

Discrete = ['Medical_History_1', 'Medical_History_10', 'Medical_History_15', 
'Medical_History_24', 'Medical_History_32']
Cate = traindf[Categorical].copy()
Cont = traindf[Continuous].copy()
Disc = traindf[Discrete].copy()



### Continuous Variables

Continuous variables are first checked and normalized if necessary to ensure that they each have a mean of zero and a standard deviation of 1.  

Second, we check for truncation.  This might occur when a variable is capped at a certain level, or if it's not possible to have a negative value and observations are lumped at zero.  In this case more information might be gained by changing the variable into a discrete variable, where our dummy variable is 1 if the variable is greater or equal to the mean, and 0 otherwise.  


In [3]:
lower_trunc = []
upper_trunc = []
Cont.is_copy = False        
for i in Continuous:
    y=traindf[i]
    name = y.name
    if y.dtype == 'float64':
        if descrip[name]['75%'] == descrip[name]['max']:
            upper_trunc.append(name)
            print upper_trunc + 'upper truncated'
        if descrip[name]['min'] == descrip[name]['25%']:
            lower_trunc.append(name)
            print name + ' lower truncated'

for i in Cont:
    if Cont[i].mean != 0:
        Cont[i] -= Cont[i].mean()
    if Cont[i].std != 1:
        Cont[i] /= Cont[i].std()

        
for i in Continuous:
    y=traindf[i]
    name = y.name
    if y.dtype == 'float64':
        if descrip[name]['75%'] == descrip[name]['max']:
            upper_trunc.append(name)
            print upper_trunc + 'upper truncated'
        if descrip[name]['min'] == descrip[name]['25%']:
            lower_trunc.append(name)
            print name + ' lower truncated'

count = []
for i in lower_trunc:
    median = descrip[i]['50%']
    nname = i + '_d'
    Cont[nname] = 0
    Cont[nname][Cont[i]>median] = 1
    Cont[nname][Cont[i].isnull()] = None

Continuous_Clean = Cont.copy()



Employment_Info_4 lower truncated
Employment_Info_4 lower truncated


C:\Users\User\AppData\Local\Enthought\Canopy\User\lib\site-packages\ipykernel\__main__.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\User\AppData\Local\Enthought\Canopy\User\lib\site-packages\ipykernel\__main__.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


For the categorical variables, we will create a new dataset with dummies for each category.  We will keep track of columns with low observations in a separate list.  The cutoff level at which we keep track is set at 100, a fairly arbitrary level, but as the list includes the observation count it is straight forward to subset the list to lower observation levels.  At this point I don't exclude any categories, because the method by which I exclude one of the categories depends on the model used, and how I choose to deal with low observation counts.  

In [4]:
import pandas as pd
Categorical_dummies = pd.DataFrame(index=Cate.index)
low_count = {}
for i in Categorical:
    test1 = pd.get_dummies(Cate[i],prefix=i)
    dummy_count = test1.sum()
    for j in range(len(dummy_count)):
        if dummy_count[j]<100:
#            print 'Note:  Low dummy count of ' + str(int(dummy_count[j])) + ' for ' + str(test1.columns[j])
            low_count[test1.columns[j]] = int(dummy_count[j])
    Categorical_dummies = pd.concat([Categorical_dummies,test1],axis=1)


Note:  Low dummy count of 54 for Product_Info_2_B1
Note:  Low dummy count of 1 for Product_Info_3_1
Note:  Low dummy count of 8 for Product_Info_3_2
Note:  Low dummy count of 2 for Product_Info_3_3
Note:  Low dummy count of 58 for Product_Info_3_4
Note:  Low dummy count of 1 for Product_Info_3_5
Note:  Low dummy count of 3 for Product_Info_3_6
Note:  Low dummy count of 48 for Product_Info_3_8
Note:  Low dummy count of 11 for Product_Info_3_9
Note:  Low dummy count of 40 for Product_Info_3_11
Note:  Low dummy count of 2 for Product_Info_3_12
Note:  Low dummy count of 1 for Product_Info_3_13
Note:  Low dummy count of 1 for Product_Info_3_16
Note:  Low dummy count of 3 for Product_Info_3_17
Note:  Low dummy count of 1 for Product_Info_3_18
Note:  Low dummy count of 1 for Product_Info_3_19
Note:  Low dummy count of 1 for Product_Info_3_20
Note:  Low dummy count of 11 for Product_Info_3_21
Note:  Low dummy count of 1 for Product_Info_3_22
Note:  Low dummy count of 10 for Product_Info_3_23
N

Now we have three datasets: **Continous_Clean**, **Categorical_dummies**, and **Disc**, matching up to the original continuous, categorical, and discrete variables.  These can be joined at this point to create a list of variables to model, or the data can be further engineered dependent on the model chosen.  For convenience sake, a list of categorical variables with low observations has also been created, **low_count**. 